In [ ]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test_certs.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [ ]:
class Сertificate(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    cluster_id = db.Column(db.Integer)
    text_from_img = db.Column(db.Text)
    user_id = db.Column(db.Integer)
    post_id = db.Column(db.Integer)
    from_id_session = db.Column(db.Integer)
    
class Session(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    session_name = db.Column(db.String(100))
    status = db.Column(db.String(100))
    
db.create_all()

In [ ]:
certificate = Сertificate(cluster_id=0, 
                        text_from_img='это текст 2', 
                        user_id=222, 
                        post_id=12, 
                        from_id_session=0)

try:
    db.session.add(certificate)
    db.session.commit()
except Exception as e:
    print(str(e))

In [ ]:
from sqlalchemy.ext.declarative import DeclarativeMeta
import json

class AlchemyEncoder(json.JSONEncoder):

    def default(self, obj):
        if isinstance(obj.__class__, DeclarativeMeta):
            # an SQLAlchemy class
            fields = {}
            for field in [x for x in dir(obj) if not x.startswith('_') and x != 'metadata']:
                data = obj.__getattribute__(field)
                try:
                    json.dumps(data) # this will fail on non-encodable values, like other classes
                    fields[field] = data
                except TypeError:
                    fields[field] = None
            # a json-encodable dict
            return fields

        return json.JSONEncoder.default(self, obj)

In [ ]:
items = Item.query.order_by(Item.user_id).all()

In [ ]:
json.loads(json.dumps(items, cls=AlchemyEncoder))